# BioE 131 Lab 4

In [1]:
#Author: Shirley Zhou

In [2]:
import sqlite3
from Bio import Entrez
from Bio import SeqIO
import matplotlib as plt
import numpy as np
import pandas as pd

import io
import os

## Table Design

For this lab assignment, we focus on three pathways: glycolysis, the citrate cycle, and the pentose phosphate pathway. We choose four enzymes in each of these pathways and search their version of gene in Drosophila, E. coli, and humans.

The tables that we would like to create in our SQL database that stores these informations are:

__Gene Table__: name, description, organism, and nucleotide sequence.

__Pathway Table__: name and description.

__Enzyme Table__: name, function, and enzyme commission (EC) number.

In [3]:
conn = sqlite3.connect('lab4.db')
c = conn.cursor()
c.execute('''CREATE TABLE gene (id TEXT, description TEXT, organism TEXT, seq TEXT);''')
c.execute('''CREATE TABLE pathway (name TEXT, description TEXT);''')
c.execute('''CREATE TABLE enzyme (name TEXT, function TEXT, ec TEXT);''')
conn.commit()

### Pathway Table

Description of pathways are easily found on the KEGG databases.

In [4]:
glycolysis_des = "process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power)"
citrate_des = "important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids"
pentose_des = "a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides"

In [5]:
c.execute('''INSERT INTO pathway (name, description)
                                 VALUES (?, ?);''',
          ("glycolysis", glycolysis_des))
c.execute('''INSERT INTO pathway (name, description)
                                 VALUES (?, ?);''',
          ("citrate cycle", citrate_des))
c.execute('''INSERT INTO pathway (name, description)
                                 VALUES (?, ?);''',
          ("Pentose phosphate", pentose_des))
conn.commit()

We can visualize the the table using `pandas.read_sql`.

In [6]:
pd.read_sql('SELECT * FROM pathway', conn)

,name,description
0,glycolysis,process of converting glucose into pyruvate an...
1,citrate cycle,important aerobic pathway for the final steps ...
2,Pentose phosphate,a process of glucose turnover that produces NA...


### Screen through databases for Enzymes

From the KEGG database, I screen through some genes in each pathway graph _(glycolysis, citrate cycle, pentose phosphate)_ that are likely to be conserved in each organism _(drosophila, E.coli, Homo Sapiens)_ .

In [7]:
glycolysis = ["phosphoglucomutase",
              "glucose-6-phosphate isomerase",
              "enolase",
              "glucokinase"]

citrate = ["aconitate hydratase",
           "pyruvate dehydrogenase",
           "succinyl-coA",
           "dihydrolipoamide dehydrogenase"]

pentose = ["ribulose-phosphate 3-epimerase", 
           "deoxyribose-phosphate aldolase", 
            "hosphogluconate dehydrogenase", 
            "phosphofructokinase"]
pw = [glycolysis, citrate, pentose]

When looking at the enzymes on the pathway maps, the enzyme commission number were labeled on the graph already.

In [8]:
glycolysis_ec = ["5.4.2.2",
                 "5.3.1.9",
                 "4.2.1.11",
                 "2.7.1.2"]

citrate_ec = ["4.2.1.3",
              "1.2.4.1",
              "6.2.1.5",
              "1.8.1.4"]

pentose_ec = ["5.1.3.1",
              "4.1.2.4",
              "1.1.1.44",
              "2.7.1.11"]

pw_ec = [glycolysis_ec, citrate_ec, pentose_ec]

Given the enzyme commision number, we could easily extract information (like enzyme function) from KEGG using `Bio.KEGG.REST` module. The only downside about this module is that the records extracted are in the form of iotextwrapper or string. Individual pieces of information can only be extracted out using string parsing (not very efficient).

In [9]:
from Bio.KEGG import REST as r

In [10]:
def enzyme_funct(ec_num):
    term = "ec:" + ec_num
    result = r.kegg_get(term).read()
    start = result.find("CLASS") + 5
    end = result.find("SYSNAME")
    function = result[start:end].replace("\n","")
    funct =" ".join(function.split())
    return funct

### Enzyme Table

Now that we have every piece of enzyme information, we can construct the enzyme table.

In [11]:
for i in np.arange(3):
    for j in np.arange(4):
        name = pw[i][j]
        ec = pw_ec[i][j]
        function = enzyme_funct(ec)
        c.execute('''INSERT INTO enzyme (name, function, ec)
                                 VALUES (?, ?, ?);''',
                 (str(name), str(function), str(ec)))
        conn.commit()


In [12]:
pd.read_sql('SELECT * FROM enzyme', conn).head()

,name,function,ec
0,phosphoglucomutase,Isomerases; Intramolecular transferases; Phosp...,5.4.2.2
1,glucose-6-phosphate isomerase,Isomerases; Intramolecular oxidoreductases; In...,5.3.1.9
2,enolase,Lyases; Carbon-oxygen lyases; Hydro-lyases,4.2.1.11
3,glucokinase,Transferases; Transferring phosphorus-containi...,2.7.1.2
4,aconitate hydratase,Lyases; Carbon-oxygen lyases; Hydro-lyases,4.2.1.3


### Gene Table

To search for these enzymes in the nucleotide database using `Bio.Entrez`, we need the corresponding gene name of those enzymes in the three different organisms. From the `kegg_get` result, we could use string manipulation to parse out the gene names given the 3-letter abbreviation of organism name. 

In [13]:
gene_name = []

for i in np.arange(3):
    for j in np.arange(4):
        term = "ec:" + pw_ec[i][j]        
        result = r.kegg_get(term).read()

        eco_start = result.find("ECO: ") + 5
        eco_parse = result[eco_start:]
        eco_gene = eco_parse[eco_parse.find("(")+1 :eco_parse.find(")")]
        gene_name.append(("e.coli", eco_gene))
    
        dme_start = result.find("DME: ") + 5
        dme_parse = result[dme_start:]
        dme_gene = dme_parse[dme_parse.find("(")+1 :dme_parse.find(")")]
        gene_name.append(("drosophila", dme_gene))

        hsa_start = result.find("HSA: ") + 5
        hsa_parse = result[hsa_start:]
        hsa_gene = hsa_parse[hsa_parse.find("(")+1 :hsa_parse.find(")")]
        gene_name.append(("homo sapiens", hsa_gene))

In [14]:
gene_name

[('e.coli', 'pgm'),
 ('drosophila', 'Pgm2b'),
 ('homo sapiens', 'PGM1'),
 ('e.coli', 'pgi'),
 ('drosophila', 'Pgi'),
 ('homo sapiens', 'GPI'),
 ('e.coli', 'eno'),
 ('drosophila', 'Eno'),
 ('homo sapiens', 'ENO1'),
 ('e.coli', 'glk'),
 ('drosophila', 'phosphorylating'),
 ('homo sapiens', 'GCK'),
 ('e.coli', 'acnB'),
 ('drosophila', 'mAcon2'),
 ('homo sapiens', 'ACO1'),
 ('e.coli', 'aceE'),
 ('drosophila', 'Pdhb'),
 ('homo sapiens', 'PDHA1'),
 ('e.coli', 'sucC'),
 ('drosophila', 'ScsbetaG'),
 ('homo sapiens', 'SUCLG2'),
 ('e.coli', 'lpd'),
 ('drosophila', 'CG7430'),
 ('homo sapiens', 'DLD'),
 ('e.coli', 'rpe'),
 ('drosophila', 'Rpe'),
 ('homo sapiens', 'RPE'),
 ('e.coli', 'deoC'),
 ('drosophila', 'Dera'),
 ('homo sapiens', 'DERA'),
 ('e.coli', 'gnd'),
 ('drosophila', 'Pgd'),
 ('homo sapiens', 'PGD'),
 ('e.coli', 'pfkB'),
 ('drosophila', 'Pfk'),
 ('homo sapiens', 'PFKL')]

Now we can define a function that takes the organism and gene name and do the entrez search for us, and return the search result as handle (genbank format) that can be later read using `SeqIO`.

In [15]:
def entrez_search(tup):
    org = tup[0]
    gene = tup[1]
    Entrez.email = 'shirleyzhou428@berkeley.edu'
    search_term = org + '[ORGN] ' + gene
    print(search_term)
    handle = Entrez.esearch(db = "nucleotide",
                            term = search_term, 
                            sort = 'relevance',
                            idtype = 'acc',
                            retmax = 1)
        
    for entry in Entrez.read(handle)['IdList']:  
        result = Entrez.efetch(db='nucleotide', id=entry, rettype='gb', retmode = 'text')
        
        record = SeqIO.read(result, "gb")
        gene_id = record.name
        descript = record.description
        seq = record.seq
        
        return (gene_id, descript, org, seq)

We can define another function that handle the reading of each record, extract different informations and put them into the gene table.

In [16]:
def add_to_sql_gene(tup):
    gene_id = tup[0]
    descript = tup[1]
    org = tup[2]
    seq = tup[3]
    
    c.execute('''INSERT INTO gene (id, description, organism, seq)
                 VALUES (?, ?, ?, ?);''',
              (str(gene_id), str(descript), str(org), str(seq)))
    conn.commit()

In [17]:
gene_list = []
for item in gene_name:
    tup = entrez_search(item)
    gene_list.append(tup)

e.coli[ORGN] pgm
drosophila[ORGN] Pgm2b
homo sapiens[ORGN] PGM1
e.coli[ORGN] pgi
drosophila[ORGN] Pgi
homo sapiens[ORGN] GPI
e.coli[ORGN] eno
drosophila[ORGN] Eno
homo sapiens[ORGN] ENO1
e.coli[ORGN] glk
drosophila[ORGN] phosphorylating
homo sapiens[ORGN] GCK
e.coli[ORGN] acnB
drosophila[ORGN] mAcon2
homo sapiens[ORGN] ACO1
e.coli[ORGN] aceE
drosophila[ORGN] Pdhb
homo sapiens[ORGN] PDHA1
e.coli[ORGN] sucC
drosophila[ORGN] ScsbetaG
homo sapiens[ORGN] SUCLG2
e.coli[ORGN] lpd
drosophila[ORGN] CG7430
homo sapiens[ORGN] DLD
e.coli[ORGN] rpe
drosophila[ORGN] Rpe
homo sapiens[ORGN] RPE
e.coli[ORGN] deoC
drosophila[ORGN] Dera
homo sapiens[ORGN] DERA
e.coli[ORGN] gnd
drosophila[ORGN] Pgd
homo sapiens[ORGN] PGD
e.coli[ORGN] pfkB
drosophila[ORGN] Pfk
homo sapiens[ORGN] PFKL


In [18]:
for item in gene_list:
    add_to_sql_gene(item)

In [19]:
pd.read_sql('SELECT * FROM gene', conn)

,id,description,organism,seq
0,FJ217664,"Escherichia coli strain 33 Pgm (pgm) gene, par...",e.coli,ATGGTGGCCCGGCTGATACCAACGTCACTAAAGTGGTGGAAGACAG...
1,NM_137148,Drosophila melanogaster phosphoglucomutase 2b ...,drosophila,CTGATATTCAAGTGTAAGTTTACTAACCAAATTTCTAGTTTGTCAC...
2,HUMPGM1A,"Human phosphoglucomutase 1 (PGM1) mRNA, comple...",homo sapiens,GGGCCGGCCGCCCCTCCGCCAGCCAAGTCCGCCGCTCTGACCCCCG...
3,AY616650,Escherichia coli strain LW1655F+ Pgi (pgi) gen...,e.coli,GCTGCCTGGCAGGCACTACAGAAACACTTCGATGAAATGAAAGACG...
4,DQ363252,"Drosophila jambulina Pgi (Pgi) gene, partial cds",drosophila,ATTGACAGCGCCAATATGTTCGGCTTCTGGGACTGGGTTGGCGGAC...
5,NM_001289789,Homo sapiens glucose-6-phosphate isomerase (GP...,homo sapiens,AATAGCCCTTACCACCAGCAGACACACATCATCTGTTGTACTTGCT...
6,MK461929,Escherichia coli strain 2009_36 plasmid p2009_...,e.coli,ATTCAGACATCAAAAAACTGTTCGGCGAGGTGGATAAGTCCTCCGG...
7,XM_001356889,Drosophila pseudoobscura pseudoobscura Eno (Dp...,drosophila,ATGGGGTGGACCACGGATCAGGAACACACACACCTGCTCATCTCCG...
8,KJ896766,Synthetic construct Homo sapiens clone ccsbBro...,homo sapiens,GTTCGTTGCAACAAATTGATGAGCAATGCTTTTTTATAATGCCAAC...
9,AFAS01000074,"Escherichia coli PCN009 contig107, whole genom...",e.coli,AGCCGAAGGACATTACCTGGTCATAACGCGGACGCGGTAACGACGC...
